In [8]:
import torch
import lib
import argparse
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
import torch.nn.functional as F
import os
import matplotlib.pyplot as plt
import torch.utils.data as data
from PIL import Image
import numpy as np
from torchvision.utils import save_image
import torch
import torch.nn.init as init
from utils import JointTransform2D, ImageToImage2D, Image2D
from metrics import jaccard_index, f1_score, LogNLLLoss,classwise_f1
from utils import chk_mkdir, Logger, MetricList
import cv2
from functools import partial
from random import randint
import timeit

In [10]:
parser = argparse.ArgumentParser(description='MedT')
parser.add_argument('-j', '--workers', default=16, type=int, metavar='N',
                    help='number of data loading workers (default: 8)')
parser.add_argument('--epochs', default=400, type=int, metavar='N',
                    help='number of total epochs to run(default: 400)')
parser.add_argument('--start-epoch', default=0, type=int, metavar='N',
                    help='manual epoch number (useful on restarts)')
parser.add_argument('-b', '--batch_size', default=1, type=int,
                    metavar='N', help='batch size (default: 1)')
parser.add_argument('--learning_rate', default=1e-3, type=float,
                    metavar='LR', help='initial learning rate (default: 0.001)')
parser.add_argument('--momentum', default=0.9, type=float, metavar='M',
                    help='momentum')
parser.add_argument('--weight-decay', '--wd', default=1e-5, type=float,
                    metavar='W', help='weight decay (default: 1e-5)')
parser.add_argument('--train_dataset', required=True, type=str)
parser.add_argument('--val_dataset', type=str)
parser.add_argument('--save_freq', type=int,default = 10)

parser.add_argument('--modelname', default='MedT', type=str,
                    help='type of model')
parser.add_argument('--cuda', default="on", type=str, 
                    help='switch on/off cuda option (default: off)')
parser.add_argument('--aug', default='off', type=str,
                    help='turn on img augmentation (default: False)')
parser.add_argument('--load', default='default', type=str,
                    help='load a pretrained model')
parser.add_argument('--save', default='default', type=str,
                    help='save the model')
parser.add_argument('--direc', default='./medt', type=str,
                    help='directory to save')
parser.add_argument('--crop', type=int, default=None)
parser.add_argument('--imgsize', type=int, default=None)
parser.add_argument('--device', default='cuda', type=str)
parser.add_argument('--gray', default='no', type=str)

_StoreAction(option_strings=['--gray'], dest='gray', nargs=None, const=None, default='no', type=<class 'str'>, choices=None, help=None, metavar=None)

In [35]:
a = torch.rand(2,5,3)
b = torch.rand(3,4)
c = torch.einsum("ik,kj->ij", [a, b])

In [36]:
c

tensor([[1.0561, 0.9630, 0.2121, 0.9440],
        [1.2736, 1.5931, 0.2059, 1.4139]])

In [37]:
c = torch.einsum("ilk,kj->ilj", [a, b])

RuntimeError: einsum(): the number of subscripts in the equation (3) does not match the number of dimensions (2) for operand 0 and no ellipsis was given

In [47]:
a = torch.rand(5,4)
b = torch.rand(5,4)

In [48]:
print(a)
print(b)
print(a*b)

tensor([[0.2489, 0.8608, 0.0839, 0.9717],
        [0.8389, 0.5973, 0.0361, 0.4765],
        [0.6156, 0.5582, 0.1748, 0.7044],
        [0.3814, 0.8125, 0.0769, 0.9378],
        [0.0756, 0.7633, 0.5888, 0.5032]])
tensor([[0.8336, 0.3617, 0.7063, 0.9085],
        [0.2821, 0.2185, 0.7458, 0.6993],
        [0.6850, 0.5674, 0.7178, 0.7828],
        [0.9968, 0.2972, 0.6626, 0.7812],
        [0.3396, 0.4766, 0.1394, 0.0646]])
tensor([[0.2075, 0.3113, 0.0592, 0.8828],
        [0.2366, 0.1305, 0.0269, 0.3332],
        [0.4217, 0.3167, 0.1254, 0.5514],
        [0.3802, 0.2415, 0.0510, 0.7326],
        [0.0257, 0.3638, 0.0821, 0.0325]])


In [72]:
a = torch.arange(0,6).reshape((1,6))
b = torch.arange(0,36).reshape((2,3,6))
c = a*b

In [62]:
a

tensor([[0, 1, 2, 3, 4, 5]])

In [75]:
b

tensor([[[ 0,  1,  2,  3,  4,  5],
         [ 6,  7,  8,  9, 10, 11],
         [12, 13, 14, 15, 16, 17]],

        [[18, 19, 20, 21, 22, 23],
         [24, 25, 26, 27, 28, 29],
         [30, 31, 32, 33, 34, 35]]])

In [73]:
c

tensor([[[  0,   1,   4,   9,  16,  25],
         [  0,   7,  16,  27,  40,  55],
         [  0,  13,  28,  45,  64,  85]],

        [[  0,  19,  40,  63,  88, 115],
         [  0,  25,  52,  81, 112, 145],
         [  0,  31,  64,  99, 136, 175]]])

In [74]:
c.shape

torch.Size([2, 3, 6])

In [77]:
b = c.sum(-1).sum

In [78]:
b

<function Tensor.sum>

In [80]:
import torch as t
a = np.ones((2,2))
a = t.Tensor(a)
print(a.shape)
 
b = t.unsqueeze(a, 1)
print(b.shape)


torch.Size([2, 2])
torch.Size([2, 1, 2])
